# Action Provider Demo: send requests directly to an AP endpoint

### 1.1 Select an Action Provider Host

In [ ]:
"""Use Diaspora AP directly through the access endpoint."""

from __future__ import annotations

import json
from random import choice
from string import ascii_uppercase
from time import time

import requests
from diaspora_event_sdk import Client as GlobusClient

# ap_endpoint = 'http://127.0.0.1:8000/'
ap_endpoint = 'https://diaspora-action-provider.ml22sevubfnks.us-east-1.cs.amazonlightsail.com'
print('AP endpoint selected:', ap_endpoint)

### 1.2 Retrieve Credential

In [ ]:
# c = GlobusClient()
# c.logout()

In [ ]:
"""A hacky way to retrieve access token."""
c = GlobusClient()
print("User's OpenID:", c.subject_openid)
tokens = c.login_manager._token_storage.get_token_data(
    '2b9d2f5c-fa32-45b5-875b-b24cd343b917',
)
access_token = tokens['access_token']
print("User's access token:", access_token)

### 1.3 Select a Topic

In [ ]:
topic = 'topic-' + c.subject_openid[-12:]
print(c.register_topic(topic))
print(c.list_topics())
print('Topic to produce/consume:', topic)

### 1.4 Set request header

In [ ]:
headers = {
    'authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
}

In [ ]:
def request_and_get_response(
    request_url: str,
    request_header: dict,
    request_data: dict,
) -> None:
    """Send API request and get response."""
    response = requests.post(
        request_url,
        headers=request_header,
        json=request_data,
    )

    # Print the response from the server
    print('Response status code:', response.status_code)
    try:
        response_content = response.content.decode('utf-8')
        parsed_content = json.loads(response_content)
        if (
            'details' in parsed_content
            and 'error' in parsed_content['details']
        ):
            error_msg = parsed_content['details']['error']
            print('Response error msg:', error_msg)
            print()
        action_id = parsed_content['action_id']
        formatted_content = json.dumps(parsed_content, indent=2)
        print('Response content:', formatted_content)
        return action_id
    except Exception:
        print('Should not happen - exception raised')
        print(response.content)

In [ ]:
def random_request_id() -> str:
    """Get a random request ID."""
    return str(''.join(choice(ascii_uppercase) for i in range(12)))

### 2.1.1 Produce Messages to AP without msg keys

In [ ]:
data = {
    'request_id': '21110',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 2.1.2: Error Case: `msgs` does not exist

In [ ]:
data = {
    'request_id': '212',
    'body': {
        'action': 'produce',
        'topic': topic,
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 2.1.3: Error Case: `msgs` is empty

In [ ]:
data = {
    'request_id': '213',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 2.1.4: Error Case: the topic does not exist (takes 10 seconds)

In [ ]:
data = {
    'request_id': '214',
    'body': {
        'action': 'produce',
        'topic': '__badtopic',
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 2.2.1 Produce Messages to AP with a Single Key

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content1': 'argonne national laboratory'},
            {'content2': 'university of chicago'},
            {'content3': 'johns hopkins university'},
        ],
        'keys': 'my-key',
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 2.3.1 Produce Messages to AP with a List of Keys

In [ ]:
data = {
    'request_id': '231',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': ['my-key1', 'my-key2', 'my-key3'],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 2.3.2 Error case: the length of `keys` does not match the length of `msgs`

In [ ]:
data = {
    'request_id': '232',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': ['my-key1', 'my-key2'],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.1.1: Consume All Messages (no `ts` nor `group_id`)

In [ ]:
data = {
    'request_id': '311',
    'body': {
        'action': 'consume',
        'topic': topic,
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.1.2: Consume Messages since the Current Timestamp (has `ts` but no `group_id`)

In [ ]:
ts_now = int(time()) * 1000

In [ ]:
data = {
    'request_id': '312',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

In [ ]:
data = {
    'request_id': '3121',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

In [ ]:
data = {
    'request_id': '312',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.1.3: Error case: the topic does not exist

In [ ]:
data = {
    'request_id': '313',
    'body': {
        'action': 'consume',
        'topic': '__badtopic',
        'ts': 1717532033372,
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.1.4: Error case: the user does not have access 

In [ ]:
data = {
    'request_id': '314',
    'body': {
        'action': 'consume',
        'topic': 'diaspora-cicd',
        'ts': 1715930522000,
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.2.1 Consume with a Prefix Filter

In [ ]:
data = {
    'request_id': '321',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.2.2 Consume with a Filter that has Multiple Conditions (cond1 AND cond2)

In [ ]:
data = {
    'request_id': '322',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
        'filters': [
            {
                'Pattern': {
                    'value': {
                        'content': [
                            {'prefix': 'hello', 'suffix': 'world2'},
                        ],
                    },
                },
            },
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.2.3 Consume with Multiple Filters (filter1 OR filter2)

In [ ]:
data = {
    'request_id': '323',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
            {'Pattern': {'value': {'content': [{'suffix': 'hello world2'}]}}},
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.2.4 Consume with Multiple Filters (repeated filters do not return repeated msgs)

In [ ]:
data = {
    'request_id': '324',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
            {'Pattern': {'value': {'content': [{'suffix': 'hello world1'}]}}},
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.2.5 Error case: Consume with a Bad Filter

In [ ]:
data = {
    'request_id': '325',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
        'filters': [
            {
                'BadPattern': {
                    'value': {'content': [{'prefix': 'hello world1'}]},
                },
            },
            {'Pattern': {'value': {'content': [{'suffix': 'hello world1'}]}}},
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.3.1 Consume with a `group_id` but no `ts` provided

In [ ]:
data = {
    'request_id': '3310',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.3.2 The same request body does not return new msgs under the same `group_id` since the last one was successful

In [ ]:
data = {
    'request_id': '33200',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

In [ ]:
data = {
    'request_id': '33111',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

In [ ]:
data = {
    'request_id': '3310',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
    },
}
request_and_get_response(f'{ap_endpoint}/run', headers, data)

### 3.3.3 Consume with a `group_id` and a `ts` (all retrieved msgs >= `ts`, if there's any)

In [ ]:
data = {
    'request_id': '333000',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
        'ts': 1715930522000,
    },
}
action_id = request_and_get_response(f'{ap_endpoint}/run', headers, data)
print(action_id)

### 4.1.1 Check Request Status

In [ ]:
print(action_id)

response = requests.get(
    f'{ap_endpoint}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 4.1.2 Check Request Status of an non-exist `action_id`
The same will happen for the action id above after TTL timeout.

In [ ]:
action_id = '1mX5FKSJTFMy'

response = requests.get(
    f'{ap_endpoint}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 4.1.3 Create an active Request, Check Status, Cancel it, and Check the Status again

In [ ]:
data = {
    'request_id': '4130',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
        'ts': 1715930522000,
    },
}
action_id = request_and_get_response(f'{ap_endpoint}/run', headers, data)
print(action_id)

In [ ]:
data = {
    'request_id': '4131',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
        'ts': 1715930522000,
    },
}
action_id = request_and_get_response(f'{ap_endpoint}/run', headers, data)
print(action_id)

Should still be ACTIVE

In [ ]:
response = requests.get(
    f'{ap_endpoint}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/cancel',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 4.1.4 Release the Request

Create a request

In [ ]:
data = {
    'request_id': '4140',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
        'ts': 1715930522000,
    },
}
action_id = request_and_get_response(f'{ap_endpoint}/run', headers, data)
print(action_id)

Error: Try to release an active request

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/release',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

Cancel the request first to release it

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/cancel',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

Check the request status

In [ ]:
response = requests.get(
    f'{ap_endpoint}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

Error: try to release the request again

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/release',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)